In [ ]:
Logic and results for rule 8831 where a comparison has to be done between a table and multiple unrelated tables.

In [4]:
import pandas as pd

sample_section47 = pd.DataFrame(
        [
            {
                "LAchildID": "child1",
                "CINdetailsID": "cinID2",
            },
            {
                "LAchildID": "child2",  # fails for having a module
                "CINdetailsID": "cinID2",
                "S47ActualStartDate": "01/01/2000",
            },
            {
                "LAchildID": "child2",
                "CINdetailsID": "cinID1",
            },
            {
                "LAchildID": "child3",
                "CINdetailsID": "cinID1",
            },
            {
                "LAchildID": "child3",
                "CINdetailsID": "cinID2",
            },
            {
                "LAchildID": "child3",
                "CINdetailsID": "cinID3",
            },
            {
                "LAchildID": "child3",
                "CINdetailsID": "cinID4",
            },
        ]
    )
sample_cin_details = pd.DataFrame(
        [
            {
                "LAchildID": "child1", # fail: has AssessmentAuthorisationDate
                "DateOfInitialCPC": pd.NA,
                "CINdetailsID": "cinID1",
                "ReferralNFA": "1",
            },
            {
                "LAchildID": "child2", # fail: has S47ActualStartDate
                "DateOfInitialCPC": pd.NA,
                "CINdetailsID": "cinID2",
                "ReferralNFA": "true",
            },
            {
                "LAchildID": "child3", # fail: has AssessmentActualStartDate
                "DateOfInitialCPC": pd.NA,
                "CINdetailsID": "cinID3",
                "ReferralNFA": "1",
            },
            {
                "LAchildID": "child4", 
                "DateOfInitialCPC": "28/05/2000",  # fails for having initial cpc
                "CINdetailsID": "cinID4",
                "ReferralNFA": "true",
            },
            {
                "LAchildID": "child3",
                "DateOfInitialCPC": "26/05/2000",
                "CINdetailsID": "cinID2",
                "ReferralNFA": "false", # ignore
            },
            {
                "LAchildID": "child3",
                "DateOfInitialCPC": "26/05/2003",
                "CINdetailsID": "cinID8",
                "ReferralNFA": "false", # ignore
            },
            {  # 6 pass
                "LAchildID": "child3",
                "DateOfInitialCPC": "14/03/2001",
                "CINdetailsID": "cinID4",
                "ReferralNFA": "false", # ignore
            },
        ]
    )
sample_ass = pd.DataFrame(
        [
            {
                "LAchildID": "child1",  # fails for having a module
                "CINdetailsID": "cinID1",
                "AssessmentAuthorisationDate": "01/01/2000",
                "AssessmentActualStartDate": pd.NA,
            },
            {
                "LAchildID": "child3",  # fails for having a module
                "CINdetailsID": "cinID3",
                "AssessmentAuthorisationDate": "01/01/2000",
                "AssessmentActualStartDate": "01/01/2000",
            },
            {
                "LAchildID": "child1",
                "CINdetailsID": "cinID2",
                "AssessmentAuthorisationDate": pd.NA,
                "AssessmentActualStartDate": pd.NA,
            },
            {
                "LAchildID": "child1",
                "CINdetailsID": "cinID3",
                "AssessmentAuthorisationDate": pd.NA,
                "AssessmentActualStartDate": pd.NA,
            },
            {
                "LAchildID": "child1",
                "CINdetailsID": "cinID4",
                "AssessmentAuthorisationDate": pd.NA,
                "AssessmentActualStartDate": pd.NA,
            },
        ]
    )

In [6]:
df_ass = sample_ass.copy()
df_ass.index.name = "ROW_ID"
df_ass.reset_index(inplace=True)

df_47 = sample_section47.copy()
df_47.index.name = "ROW_ID"
df_47.reset_index(inplace=True)

df_cin = sample_cin_details.copy()
df_cin.index.name = "ROW_ID"
df_cin.reset_index(inplace=True)

print(df_ass)
print(df_cin)
print(df_47)

   ROW_ID LAchildID CINdetailsID AssessmentAuthorisationDate  \
0       0    child1       cinID1                  01/01/2000   
1       1    child3       cinID3                  01/01/2000   
2       2    child1       cinID2                        <NA>   
3       3    child1       cinID3                        <NA>   
4       4    child1       cinID4                        <NA>   

  AssessmentActualStartDate  
0                      <NA>  
1                01/01/2000  
2                      <NA>  
3                      <NA>  
4                      <NA>  
   ROW_ID LAchildID DateOfInitialCPC CINdetailsID ReferralNFA
0       0    child1             <NA>       cinID1           1
1       1    child2             <NA>       cinID2        true
2       2    child3             <NA>       cinID3           1
3       3    child4       28/05/2000       cinID4        true
4       4    child3       26/05/2000       cinID2       false
5       5    child3       26/05/2003       cinID8       false
6

In [9]:
ReferralNFA = "ReferralNFA"
trueor1 = ["true", "1"]
df_cin = df_cin[df_cin[ReferralNFA].isin(trueor1)]
print(df_cin)

   ROW_ID LAchildID DateOfInitialCPC CINdetailsID ReferralNFA
0       0    child1             <NA>       cinID1           1
1       1    child2             <NA>       cinID2        true
2       2    child3             <NA>       cinID3           1
3       3    child4       28/05/2000       cinID4        true


In [11]:
DateOfInitialCPC = "DateOfInitialCPC"
S47ActualStartDate = "S47ActualStartDate"
CINdetailsID = "CINdetailsID"
LAchildID = "LAchildID"

# Check columns in Section47 table
df_cin_47 = df_cin.merge(df_47, on=["LAchildID", "CINdetailsID"], how="left", suffixes=["_cin", "_47"])
# filter out rows that have an S47ActualStartDate or DateOfInitialCPC
condition_1 = (df_cin_47[DateOfInitialCPC].notna() | df_cin_47[S47ActualStartDate].notna())
df_cin_47 = df_cin_47[condition_1]
df_cin_47["ERROR_ID"] = tuple(zip(df_cin_47[LAchildID], df_cin_47[CINdetailsID]))
df_47_issues = (
    df_47.merge(df_cin_47, left_on="ROW_ID", right_on="ROW_ID_47")
    .groupby("ERROR_ID", group_keys=False)["ROW_ID"]
    .apply(list)
    .reset_index()
)
cin_issues_47 = (
    df_cin.merge(df_cin_47, left_on="ROW_ID", right_on="ROW_ID_cin")
    .groupby("ERROR_ID", group_keys=False)["ROW_ID"]
    .apply(list)
    .reset_index()
)
print(cin_issues_47)


           ERROR_ID ROW_ID
0  (child2, cinID2)    [1]
1  (child4, cinID4)    [3]


In [14]:
CINdetailsID = "CINdetailsID"
LAchildID = "LAchildID"
AssessmentActualStartDate = "AssessmentActualStartDate"
AssessmentAuthorisationDate = "AssessmentAuthorisationDate"

# Check columns in Assessments table
df_cin_ass = df_cin.merge(
    df_ass,
    on=["LAchildID", "CINdetailsID"],
    how="left",
    suffixes=["_cin", "_ass"], 
)    
# filter out rows that have an AssessmentActualStartDate or AssessmentAuthorisationDate
condition_2 = df_cin_ass[AssessmentActualStartDate].notna()
condition_3 = df_cin_ass[AssessmentAuthorisationDate].notna()
df_cin_ass = df_cin_ass[condition_2 | condition_3]
df_cin_ass["ERROR_ID"] = tuple(zip(df_cin_ass[LAchildID], df_cin_ass[CINdetailsID]))
df_ass_issues = (
    df_ass.merge(df_cin_ass, left_on="ROW_ID", right_on="ROW_ID_ass")
    .groupby("ERROR_ID", group_keys=False)["ROW_ID"]
    .apply(list)
    .reset_index()
)    
cin_issues_ass = (
    df_cin.merge(df_cin_ass, left_on="ROW_ID", right_on="ROW_ID_cin")
    .groupby("ERROR_ID", group_keys=False)["ROW_ID"]
    .apply(list)
    .reset_index()
)

print(cin_issues_ass)

           ERROR_ID ROW_ID
0  (child1, cinID1)    [0]
1  (child3, cinID3)    [2]


In [15]:
df_cin_issues = pd.concat([cin_issues_47, cin_issues_ass])
print(df_cin_issues)

           ERROR_ID ROW_ID
0  (child2, cinID2)    [1]
1  (child4, cinID4)    [3]
0  (child1, cinID1)    [0]
1  (child3, cinID3)    [2]


In [16]:
df_cin_issues.drop_duplicates("ERROR_ID")
print(df_cin_issues)

           ERROR_ID ROW_ID
0  (child2, cinID2)    [1]
1  (child4, cinID4)    [3]
0  (child1, cinID1)    [0]
1  (child3, cinID3)    [2]
